In [ ]:
'''
00_json_to_csv.ipynb

목적: 원본 JSON(NODE_LIST)을 스트리밍으로 CSV/Parquet로 변환

입력: data/raw/*.json 또는 Drive의 원본 json

출력: data/interim/papers.parquet (+ 공유용 outputs/tables/papers.csv)

실행 순서: 이 노트북 → 10_preprocess → 20_kw_extract

주의: 대용량이라 ijson 필요, 로컬 업로드 대신 Drive 경로 권장
'''

In [ ]:
import importlib.util

if importlib.util.find_spec("ijson") is None:
    !pip -q install ijson

import ijson

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import os
out_dir = "/content/drive/MyDrive/SSU_Datathon2025"
os.makedirs(out_dir, exist_ok=True)
print(out_dir)

/content/drive/MyDrive/SSU_Datathon2025


In [ ]:
import csv
import ijson
from google.colab import files

uploaded = files.upload()
json_path = list(uploaded.keys())[0]
csv_path = json_path.replace(".json", ".csv")

with open(json_path, "rb") as f:
    items = ijson.items(f, "NODE_LIST.item")

    first = next(items, None)
    if first is None:
        raise ValueError("NODE_LIST가 비어있어요")

    fieldnames = list(first.keys())

    with open(csv_path, "w", newline="", encoding="utf-8-sig") as out:
        writer = csv.DictWriter(out, fieldnames=fieldnames, extrasaction="ignore")
        writer.writeheader()
        writer.writerow(first)

        for item in items:
            row = {k: item.get(k, "") for k in fieldnames}
            writer.writerow(row)

print("saved:", csv_path)

KeyboardInterrupt: 

In [ ]:
import pandas as pd

df = pd.read_csv(csv_path, nrows=20)
df.head()

NameError: name 'csv_path' is not defined

In [ ]:
import pandas as pd

csv_path = "SSU_Datathon2025_공학분야_62199.csv"
papers = pd.read_csv(csv_path, low_memory=False)

papers.to_csv("papers.csv", index=False, encoding="utf-8-sig")
papers.to_parquet("papers.parquet", index=False)

print("saved: papers.csv, papers.parquet")

saved: papers.csv, papers.parquet


In [ ]:
papers.to_csv(f"{out_dir}/papers.csv", index=False, encoding="utf-8-sig")
papers.to_parquet(f"{out_dir}/papers.parquet", index=False)
print("saved to drive:", out_dir)